### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
file_to_load = "Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
list_of_players = purchase_data['SN'].value_counts()
len(list_of_players)

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
number_of_items = len(purchase_data['Item ID'].unique())
average_price = purchase_data['Price'].mean()
number_of_purchases = len(purchase_data['Purchase ID'].unique())
total_revenue = purchase_data['Price'].sum()

In [4]:
purchasing_analysis = [{"Average Price": average_price, "Number of Unique Items": number_of_items, 
                        "Number of Purchases": number_of_purchases, "Total Revenue": total_revenue}]
df_purchasing_analysis = pd.DataFrame(purchasing_analysis)

# Applying formats
df_purchasing_analysis["Average Price"] = df_purchasing_analysis["Average Price"].map("${:.2f}".format)
df_purchasing_analysis["Total Revenue"] = df_purchasing_analysis["Total Revenue"].map("${:,}".format)
df_purchasing_analysis

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$3.05,780,183,"$2,379.77"


## Gender Demographics

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
# Group by username and gender to eliminate duplicate purchases
group_by_username = purchase_data.groupby(['Gender', 'SN'])
users = group_by_username['SN'].count()

# Make new df with merged user names
gender_summary = pd.DataFrame(users)
gender_summary.head(2)

SN
Gender SN               
Female Adastirin33     1
       Aerithllora36   2

In [6]:
# Group by Gender new dataframe to see number of users of each gender
group_by_gender = gender_summary.groupby(['Gender'])
gender_counts = group_by_gender.count()
gender_counts

,SN
Gender,
Female,81
Male,484
Other / Non-Disclosed,11


In [7]:
percent_of_players = [round(x/len(users)*100, 2) for x in gender_counts['SN']]
percent_of_players

[14.06, 84.03, 1.91]

In [8]:
# Total number of Female players 81, not 113
gender_summary = pd.DataFrame({"Percentage of Players": percent_of_players, 
                               "Total Count": gender_counts['SN']})
gender_summary 

,Percentage of Players,Total Count
Gender,,
Female,14.06,81
Male,84.03,484
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, etc. by gender


* For normalized purchasing, divide total purchase value by purchase count, by gender


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [18]:
group_by_gender = purchase_data.groupby(['Gender'])
total_count = group_by_gender['Item ID'].count()
average_p = group_by_gender['Price'].mean()
total_value = group_by_gender['Price'].sum()

# Make a nested list comprehensions to iterate both through "Purchase value" and "Purchase count"
normalized_p = [(value/count) for value, count in zip(total_value, total_count)]

gender_summary = pd.DataFrame({"Purchase Count": total_count, "Average Purchase Price": average_p, 
                               "Total Purchase Value": total_value, "Normalized Totals": normalized_p})

# Applying formats
gender_summary["Average Purchase Price"] = gender_summary["Average Purchase Price"].map("${:.2f}".format)
gender_summary["Normalized Totals"] = gender_summary["Normalized Totals"].map("${:.2f}".format)
gender_summary["Total Purchase Value"] = gender_summary["Total Purchase Value"].map("${:,.2f}".format)


gender_summary


,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
Gender,,,,
Female,$3.20,$3.20,113,$361.94
Male,$3.02,$3.02,652,"$1,967.64"
Other / Non-Disclosed,$3.35,$3.35,15,$50.19


## Age Demographics

In [311]:
age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"], age_bins, labels = group_names)

# Group by Age groups
group_by_age = purchase_data.groupby(['Age Groups'])

# Calculate the numbers and percentages by age group
age_purchases = group_by_age['SN'].count()
percent_of_purch = [round(x/sum(age_purchases)*100, 2) for x in age_purchases]

# Create a summary data frame to hold the results
age_summary = pd.DataFrame({"Percentage of Players": percent_of_purch, "Total Count": age_purchases})

age_summary

,Percentage of Players,Total Count
Age Groups,,
<10,2.95,23
10-14,3.59,28
15-19,17.44,136
20-24,46.79,365
25-29,12.95,101
30-34,9.36,73
35-39,5.26,41
40+,1.67,13


## Purchasing Analysis (Age)

In [318]:
age_summary_p = group_by_age['Price'].mean()
age_summary_v = group_by_age['Price'].sum()

# Make a nested list comprehensions to iterate both through "Purchase value" and "Purchase count"
normalized_p = [(value/count) for value, count in zip(age_summary_v, age_purchases)]

purchasing_analysis_age = pd.DataFrame({"Purchase Count": age_purchases, "Average Purchase Price": age_summary_p, 
                                        "Total Purchase Value": age_summary_v, "Normalized Totals": normalized_p})

# Applying formats
purchasing_analysis_age["Average Purchase Price"] = purchasing_analysis_age["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_age["Normalized Totals"] = purchasing_analysis_age["Normalized Totals"].map("${:.2f}".format)
purchasing_analysis_age["Total Purchase Value"] = purchasing_analysis_age["Total Purchase Value"].map("${:,.2f}".format)

purchasing_analysis_age

,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
Age Groups,,,,
<10,$3.35,$3.35,23,$77.13
10-14,$2.96,$2.96,28,$82.78
15-19,$3.04,$3.04,136,$412.89
20-24,$3.05,$3.05,365,"$1,114.06"
25-29,$2.90,$2.90,101,$293.00
30-34,$2.93,$2.93,73,$214.00
35-39,$3.60,$3.60,41,$147.67
40+,$2.94,$2.94,13,$38.24


## Top Spenders

* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the summary data frame

In [326]:
group_by_username = purchase_data.groupby(['SN'])
top_count = group_by_username['SN'].count()
top_price = group_by_username['Price'].mean()
top_value = group_by_username['Price'].sum()

top_spenders = pd.DataFrame({"Purchase Count": top_count, "Average Purchase Price": top_price, "Total Purchase Value": top_value})
sorted_top = top_spenders.sort_values(["Total Purchase Value"], ascending=False)

# Applying formats
sorted_top["Average Purchase Price"] = sorted_top["Average Purchase Price"].map("${:.2f}".format)
sorted_top["Total Purchase Value"] = sorted_top["Total Purchase Value"].map("${:,.2f}".format)

sorted_top.head(5)

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Lisosia93,$3.79,5,$18.96
Idastidru52,$3.86,4,$15.45
Chamjask73,$4.61,3,$13.83
Iral74,$3.40,4,$13.62
Iskadarya95,$4.37,3,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [331]:
group_by_item = purchase_data.groupby(['Item ID','Item Name'])
item_count = group_by_item['SN'].count()
item_price = group_by_item['Price'].mean()
item_value = group_by_item['Price'].sum()

top_items = pd.DataFrame({"Purchase Count": item_count, "Item Price": item_price, "Total Purchase Value": item_value})
sorted_top_items = top_items.sort_values(["Purchase Count"], ascending=False)

# Applying formats
sorted_top_items["Item Price"] = sorted_top_items["Item Price"].map("${:.2f}".format)
sorted_top_items["Total Purchase Value"] = sorted_top_items["Total Purchase Value"].map("${:,.2f}".format)

sorted_top_items.head(5)


,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
145,Fiery Glass Crusader,$4.58,9,$41.22
108,"Extraction, Quickblade Of Trembling Hands",$3.53,9,$31.77
82,Nirvana,$4.90,9,$44.10
19,"Pursuit, Cudgel of Necromancy",$1.02,8,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [334]:
sorted_top_items_value = top_items.sort_values(["Total Purchase Value"], ascending=False)

# Applying formats
sorted_top_items_value["Item Price"] = sorted_top_items_value["Item Price"].map("${:.2f}".format)
sorted_top_items_value["Total Purchase Value"] = sorted_top_items_value["Total Purchase Value"].map("${:,.2f}".format)

sorted_top_items_value.head(5)

,,Item Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
82,Nirvana,$4.90,9,$44.10
145,Fiery Glass Crusader,$4.58,9,$41.22
92,Final Critic,$4.88,8,$39.04
103,Singed Scalpel,$4.35,8,$34.80
